### import bibliotek

In [44]:
import requests
from bs4 import BeautifulSoup
import json
import os

In [42]:
with open('./headers.json', 'r') as jf:

    headers=json.load(jf)


Pobranie ze strony Ceneo.pl opinii o  konkretnym produkcie



In [ ]:
product_id="/176018202"
next_page=f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_opinios=[]
while next_page:
    response = requests.get(next_page, headers= headers)
    print(next_page)
    if response.status_code==200:
        page_dom=  BeautifulSoup(response.text, "html.parser")
        opinions= page_dom.select('div.js_product-review:not(.user-post--highlight)')
        print(len(opinions))
        for opinion in opinions:
                try:
                    single_opinion={
                        'review_id': opinion['data-entry-id'],
                        'author': opinion.select_one('span.user-post__author-name').text.strip(),
                        'recomendation':opinion.select_one('span.user-post__author-recomendation > em').text.strip(),
                        'stars': opinion.select_one('span.user-post__score-count').text.strip(),
                        'content': opinion.select_one('div.user-post__text').text.strip(),
                        'pros': [p.text.strip() for p in opinion.select('div.review-feature__item--positive')],
                        'cons': [c.text.strip() for c in opinion.select('div.review-feature__item--negative')],
                        'likes': opinion.select_one('button.vote-yes > span').text.strip(),
                        'dislikes': opinion.select_one('button.vote-no > span').text.strip(),
                        'publish_date':opinion.select_one('span.user-post__published > time:nth-child(1)')['datetime'].strip(),
                        'purchase_date': opinion.select_one('span.user-post__published > time:nth-child(2)')['datetime'].strip()
                    }
                    all_opinios.append(single_opinion)
                except (AttributeError, TypeError): 
                    pass
        
        try:
            next_page="https://www.ceneo.pl"+page_dom.select_one('a.pagination__next')['href']
                
        except TypeError:
            next_page=None
print(all_opinios)

### zapisanie wszystkich opinii w bazie danych


In [51]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [58]:
with open(f"./opinions/{product_id}.json", "w", encoding="UTF-8") as jf:
    json.dump(all_opinios, jf, indent= 4, ensure_ascii=False)